<a href="https://colab.research.google.com/github/ramonVDAKKER/teaching-data-science-emas/blob/main/notebooks/short_standard_demo_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Snippets for in-class demo and discussion of image classification and transfer learning



## 0. Imports

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt
from PIL import Image
from urllib import request
from io import BytesIO
from keras.models import Sequential
from keras.models import Model
from keras import optimizers
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
import tensorflow_hub as hub
import tensorflow as tf

## 1. Load model

We load the InceptionV3 model which is included in the keras package.

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
model = InceptionV3()
model.summary()

# 2. Using the model

Choose a photo.

In [ ]:
url = "https://kc-shertogenbosch.nl/wp-content/uploads/2018/11/Nederlandse-Schapendoes.jpg"
res = request.urlopen(url).read()
image = Image.open(BytesIO(res)).resize((299, 299))
plt.imshow(image)

Transform the image to numerical data.

In [ ]:
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)
print(image)

Use the model to classify the image.


In [ ]:
model.predict(image)

In [ ]:
predicted_label = decode_predictions(model.predict(image))
for item in predicted_label[0]:
  print(f"Label {item[1]} has probability: {item[2]}.")

## 3. Transfer learning

Code partially taken from https://www.tensorflow.org/hub/tutorials/tf2_image_retraining

Load dataset with flower images:

In [ ]:
data_dir = tf.keras.utils.get_file("flower_photos",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz", untar=True)
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255)
def build_dataset(subset):
  dataset = tf.keras.preprocessing.image_dataset_from_directory(data_dir,
      validation_split=.30,
      subset=subset,
      label_mode="categorical",
      seed=123,
      image_size=(299, 299),
      batch_size=1)
  size = dataset.cardinality().numpy()
  classes = tuple(dataset.class_names)
  dataset = dataset.unbatch().batch(32)
 # dataset = dataset.repeat()
  dataset = dataset.map(lambda images, labels: (normalization_layer(images), labels))
  #dataset = dataset.map(lambda images, labels: (preprocess_input(images), labels))
  return dataset, classes, size
train_data, class_names, train_size = build_dataset("training")
validation_data, _, validation_size = build_dataset("validation")

Let us inspect a few images:

In [ ]:
x, y = next(iter(validation_data))
k = 5
predicted_label = decode_predictions(model.predict(x[0 : k, :, :, :]))
for i in range(0, k):
  image = x[i, :, :, :]
  true_index = np.argmax(y[i])
  plt.imshow(image)
  plt.axis('off')
  plt.title(class_names[true_index])
  plt.show()

InceptionV3, with our setting to consider 1,000 classes, is not specific enough to distinguish the flowers:

In [ ]:
for item in predicted_label[i]:
  print(f"Label {item[1]} has probability: {item[2]}.")

Using the small (!) flower dataset and InceptionV3 we will train a neural network. First we specify the model:

In [ ]:
base_model = InceptionV3(include_top=False, input_shape=(299, 299) + (3,))
base_model.trainable = False
model = Sequential([
    tf.keras.layers.InputLayer(input_shape=(299, 299) + (3,)),
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5", trainable=False),
    tf.keras.layers.Dropout(rate=0.4),
    tf.keras.layers.Dense(5, kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build([None, 299, 299, 3])
model.summary()

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])
steps_per_epoch = train_size // 32
validation_steps = validation_size // 32
hist = model.fit(train_data, epochs=50, steps_per_epoch=steps_per_epoch, validation_data=validation_data).history